In [1]:
import os
import random
import numpy as np
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Global counter for compression tracking
compression_counter = 0

def jpeg_compress_np(image, quality):
    """Compress a NumPy image using JPEG at the given quality"""
    global compression_counter
    
    # OpenCV expects images in BGR format
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # Explicitly cast quality to int
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), int(quality)]
    success, encimg = cv2.imencode('.jpg', image_bgr, encode_param)
    if success:
        compression_counter += 1
        if compression_counter % 10000 == 0:
            print(f"Compressed {compression_counter} images")
        decimg = cv2.imdecode(encimg, cv2.IMREAD_COLOR)
        # Convert back to RGB
        decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)
        return decimg
    else:
        return image

In [4]:
def create_compressed_dataset(data_dir, quality, target_size=(224, 224)):
    """
    Load all images, compress them, and create train/val splits
    Returns: train_data, val_data where each is a list of (image_tensor, label) tuples
    """
    # Define transforms
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Get all classes
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
    
    print(f"Found {len(classes)} classes")
    
    # Collect all image paths and labels
    all_samples = []
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    all_samples.append((
                        os.path.join(class_dir, img_name),
                        class_to_idx[class_name]
                    ))
    
    print(f"Found {len(all_samples)} images")
    
    # Shuffle the dataset
    random.shuffle(all_samples)
    
    # Split into train and validation
    split_idx = int(0.8 * len(all_samples))
    train_samples = all_samples[:split_idx]
    val_samples = all_samples[split_idx:]
    
    print(f"Split: {len(train_samples)} training, {len(val_samples)} validation")
    
    # Process all images
    train_data = []
    val_data = []
    
    print("Processing training images...")
    for img_path, label in train_samples:
        # Load and resize image
        img = Image.open(img_path).convert('RGB')
        img = img.resize(target_size)
        img_np = np.array(img)
        
        # Compress image
        compressed_img = jpeg_compress_np(img_np, quality)
        
        # Convert to tensor and normalize
        img_tensor = transform(Image.fromarray(compressed_img))
        
        train_data.append((img_tensor, label))
    
    print("Processing validation images...")
    for img_path, label in val_samples:
        # Load and resize image
        img = Image.open(img_path).convert('RGB')
        img = img.resize(target_size)
        img_np = np.array(img)
        
        # Compress image
        compressed_img = jpeg_compress_np(img_np, quality)
        
        # Convert to tensor and normalize
        img_tensor = transform(Image.fromarray(compressed_img))
        
        val_data.append((img_tensor, label))
    
    print(f"Processed {len(train_data)} training images and {len(val_data)} validation images")
    return train_data, val_data

In [5]:
def create_batches(data, batch_size):
    """Create batches from a list of (image, label) tuples"""
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        images = torch.stack([item[0] for item in batch])
        labels = torch.tensor([item[1] for item in batch])
        yield images, labels

In [6]:
def train_model(data_dir, quality, model_save_path, batch_size=32, epochs=5):
    """Train a model using manually created batches instead of DataLoader"""
    # Create dataset
    train_data, val_data = create_compressed_dataset(data_dir, quality)
    
    # Load pre-trained ResNet50 model
    model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
    
    # Freeze base model layers
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace the last fully connected layer
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, 1000)  # 1000 classes for ImageNet
    
    # Move model to GPU if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(f"Model moved to {device}")
    
    # Define loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-4)
    
    # Training loop
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Training phase
        model.train()
        running_loss = 0.0
        train_batches = 0
        
        # Shuffle training data at the start of each epoch
        random.shuffle(train_data)
        
        # Manual batching
        for i, (inputs, labels) in enumerate(create_batches(train_data, batch_size)):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            train_batches += 1
            
            if i % 10 == 9:  # Print every 10 batches
                print(f"Batch {i+1}, Loss: {running_loss/10:.4f}")
                running_loss = 0.0
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in create_batches(val_data, batch_size):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_acc = correct / total
        print(f"Validation Accuracy: {val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved to {model_save_path}")
    
    return model

In [10]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 50
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_low.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 20
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 60000 images
Compressed 70000 images
Compressed 80000 images
Compressed 90000 images
Processing validation images...
Compressed 100000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/20
Batch 10, Loss: 6.9176
Batch 20, Loss: 6.9051
Batch 30, Loss: 6.8998
Batch 40, Loss: 6.9084
Batch 50, Loss: 6.9269
Batch 60, Loss: 6.8722
Batch 70, Loss: 6.8421
Batch 80, Loss: 6.8482
Batch 90, Loss: 6.8076
Batch 100, Loss: 6.8339
Batch 110, Loss: 6.7494
Batch 120, Loss: 6.7622
Batch 130, Loss: 6.7424
Batch 140, Loss: 6.7611
Batch 150, Loss: 6.7304
Batch 160, Loss: 6.6904
Batch 170, Loss: 6.6336
Batch 180, Loss: 6.6595
Batch 190, Loss: 6.5650
Batch 200, Loss: 6.5483
Batch 210, Loss: 6.6188
Batch 220, Loss: 6.5854
Batch 230, Loss: 6.5505
Batch 240, Loss: 6.5215
Batch 250, Loss: 6.5202
Batch 260, Loss: 6.5197
Batch 270, Loss: 6.5279
Batch 2

In [34]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 90
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_mid.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 20
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 10000 images
Compressed 20000 images
Compressed 30000 images
Compressed 40000 images
Processing validation images...
Compressed 50000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/20
Batch 10, Loss: 6.9280
Batch 20, Loss: 6.9059
Batch 30, Loss: 6.8993
Batch 40, Loss: 6.8991
Batch 50, Loss: 6.8636
Batch 60, Loss: 6.8510
Batch 70, Loss: 6.8486
Batch 80, Loss: 6.7655
Batch 90, Loss: 6.7455
Batch 100, Loss: 6.7923
Batch 110, Loss: 6.8131
Batch 120, Loss: 6.7612
Batch 130, Loss: 6.7031
Batch 140, Loss: 6.6935
Batch 150, Loss: 6.6599
Batch 160, Loss: 6.6385
Batch 170, Loss: 6.6449
Batch 180, Loss: 6.6096
Batch 190, Loss: 6.6511
Batch 200, Loss: 6.5490
Batch 210, Loss: 6.5596
Batch 220, Loss: 6.5411
Batch 230, Loss: 6.5462
Batch 240, Loss: 6.5000
Batch 250, Loss: 6.4765
Batch 260, Loss: 6.4765
Batch 270, Loss: 6.4407
Batch 28

In [12]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 75
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_high.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 20
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 160000 images
Compressed 170000 images
Compressed 180000 images
Compressed 190000 images
Processing validation images...
Compressed 200000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/20
Batch 10, Loss: 6.9206
Batch 20, Loss: 6.9029
Batch 30, Loss: 6.8989
Batch 40, Loss: 6.9031
Batch 50, Loss: 6.8917
Batch 60, Loss: 6.8596
Batch 70, Loss: 6.8678
Batch 80, Loss: 6.8237
Batch 90, Loss: 6.7961
Batch 100, Loss: 6.7836
Batch 110, Loss: 6.8240
Batch 120, Loss: 6.7963
Batch 130, Loss: 6.7564
Batch 140, Loss: 6.6729
Batch 150, Loss: 6.6805
Batch 160, Loss: 6.6250
Batch 170, Loss: 6.6474
Batch 180, Loss: 6.6114
Batch 190, Loss: 6.6395
Batch 200, Loss: 6.5615
Batch 210, Loss: 6.5647
Batch 220, Loss: 6.5389
Batch 230, Loss: 6.5203
Batch 240, Loss: 6.5177
Batch 250, Loss: 6.4817
Batch 260, Loss: 6.4511
Batch 270, Loss: 6.4882
Bat

# New Saves for Epoch 50

In [7]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 60
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_60_epoch50.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 50
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 10000 images
Compressed 20000 images
Compressed 30000 images
Compressed 40000 images
Processing validation images...
Compressed 50000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/50
Batch 10, Loss: 6.9138
Batch 20, Loss: 6.9225
Batch 30, Loss: 6.9037
Batch 40, Loss: 6.8967
Batch 50, Loss: 6.9034
Batch 60, Loss: 6.9013
Batch 70, Loss: 6.8531
Batch 80, Loss: 6.8783
Batch 90, Loss: 6.8202
Batch 100, Loss: 6.8196
Batch 110, Loss: 6.7629
Batch 120, Loss: 6.7711
Batch 130, Loss: 6.7648
Batch 140, Loss: 6.7009
Batch 150, Loss: 6.7019
Batch 160, Loss: 6.6553
Batch 170, Loss: 6.6759
Batch 180, Loss: 6.6337
Batch 190, Loss: 6.6344
Batch 200, Loss: 6.6111
Batch 210, Loss: 6.5473
Batch 220, Loss: 6.5590
Batch 230, Loss: 6.6196
Batch 240, Loss: 6.5455
Batch 250, Loss: 6.5094
Batch 260, Loss: 6.5120
Batch 270, Loss: 6.4790
Batch 28

In [8]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 70
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_70_epoch50.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 50
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 60000 images
Compressed 70000 images
Compressed 80000 images
Compressed 90000 images
Processing validation images...
Compressed 100000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/50
Batch 10, Loss: 6.9144
Batch 20, Loss: 6.9070
Batch 30, Loss: 6.9050
Batch 40, Loss: 6.8923
Batch 50, Loss: 6.9101
Batch 60, Loss: 6.8707
Batch 70, Loss: 6.8240
Batch 80, Loss: 6.7810
Batch 90, Loss: 6.8421
Batch 100, Loss: 6.8058
Batch 110, Loss: 6.7462
Batch 120, Loss: 6.7739
Batch 130, Loss: 6.7134
Batch 140, Loss: 6.7209
Batch 150, Loss: 6.7100
Batch 160, Loss: 6.6972
Batch 170, Loss: 6.6651
Batch 180, Loss: 6.6243
Batch 190, Loss: 6.5875
Batch 200, Loss: 6.5938
Batch 210, Loss: 6.5774
Batch 220, Loss: 6.5768
Batch 230, Loss: 6.5101
Batch 240, Loss: 6.4905
Batch 250, Loss: 6.4995
Batch 260, Loss: 6.5096
Batch 270, Loss: 6.4655
Batch 2

In [9]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 80
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_80_epoch50.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 50
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 110000 images
Compressed 120000 images
Compressed 130000 images
Compressed 140000 images
Processing validation images...
Compressed 150000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/50
Batch 10, Loss: 6.9079
Batch 20, Loss: 6.8925
Batch 30, Loss: 6.9215
Batch 40, Loss: 6.8695
Batch 50, Loss: 6.8715
Batch 60, Loss: 6.8335
Batch 70, Loss: 6.8658
Batch 80, Loss: 6.7860
Batch 90, Loss: 6.7811
Batch 100, Loss: 6.8322
Batch 110, Loss: 6.7559
Batch 120, Loss: 6.7665
Batch 130, Loss: 6.7254
Batch 140, Loss: 6.7457
Batch 150, Loss: 6.6960
Batch 160, Loss: 6.6328
Batch 170, Loss: 6.6559
Batch 180, Loss: 6.6295
Batch 190, Loss: 6.5867
Batch 200, Loss: 6.5820
Batch 210, Loss: 6.5605
Batch 220, Loss: 6.5990
Batch 230, Loss: 6.5129
Batch 240, Loss: 6.5489
Batch 250, Loss: 6.4804
Batch 260, Loss: 6.4842
Batch 270, Loss: 6.4567
Bat

In [10]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 90
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_90_epoch50.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 50
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 160000 images
Compressed 170000 images
Compressed 180000 images
Compressed 190000 images
Processing validation images...
Compressed 200000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/50
Batch 10, Loss: 6.9135
Batch 20, Loss: 6.8987
Batch 30, Loss: 6.8786
Batch 40, Loss: 6.8838
Batch 50, Loss: 6.9149
Batch 60, Loss: 6.8761
Batch 70, Loss: 6.8626
Batch 80, Loss: 6.8190
Batch 90, Loss: 6.8109
Batch 100, Loss: 6.7941
Batch 110, Loss: 6.7745
Batch 120, Loss: 6.7928
Batch 130, Loss: 6.6793
Batch 140, Loss: 6.6926
Batch 150, Loss: 6.6981
Batch 160, Loss: 6.6509
Batch 170, Loss: 6.5834
Batch 180, Loss: 6.6224
Batch 190, Loss: 6.6012
Batch 200, Loss: 6.5657
Batch 210, Loss: 6.5460
Batch 220, Loss: 6.5546
Batch 230, Loss: 6.5030
Batch 240, Loss: 6.4481
Batch 250, Loss: 6.4786
Batch 260, Loss: 6.4600
Batch 270, Loss: 6.4600
Bat

In [11]:
if __name__ == "__main__":
    # Update this path to point to your local dataset
    data_dir = "C:/Users/CSE IIT BHILAI/Desktop/Adversarial ML/jpeg-defense/data/temp/val"  # For Windows
    
    # Choose a JPEG quality for vaccination
    quality = 100
    
    # Path where the fine-tuned model will be saved
    model_save_path = "resnet50_100_epoch50.pth"
    
    # Define training parameters
    batch_size = 32
    epochs = 50
    
    # Train the model
    model_low = train_model(data_dir, quality, model_save_path, batch_size, epochs)
    
    print("Model training complete. Fine-tuned model saved to:", model_save_path)

Found 1000 classes
Found 50000 images
Split: 40000 training, 10000 validation
Processing training images...
Compressed 210000 images
Compressed 220000 images
Compressed 230000 images
Compressed 240000 images
Processing validation images...
Compressed 250000 images
Processed 40000 training images and 10000 validation images
Model moved to cuda:0
Epoch 1/50
Batch 10, Loss: 6.9127
Batch 20, Loss: 6.9068
Batch 30, Loss: 6.8974
Batch 40, Loss: 6.8943
Batch 50, Loss: 6.8750
Batch 60, Loss: 6.8422
Batch 70, Loss: 6.8731
Batch 80, Loss: 6.8207
Batch 90, Loss: 6.8365
Batch 100, Loss: 6.7733
Batch 110, Loss: 6.7512
Batch 120, Loss: 6.7617
Batch 130, Loss: 6.7225
Batch 140, Loss: 6.6932
Batch 150, Loss: 6.6632
Batch 160, Loss: 6.6194
Batch 170, Loss: 6.6516
Batch 180, Loss: 6.6027
Batch 190, Loss: 6.5669
Batch 200, Loss: 6.5391
Batch 210, Loss: 6.6000
Batch 220, Loss: 6.5356
Batch 230, Loss: 6.4672
Batch 240, Loss: 6.4668
Batch 250, Loss: 6.4873
Batch 260, Loss: 6.4568
Batch 270, Loss: 6.4434
Bat